In [2]:
import os
import sys
import numpy as np
import pandas as pd
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import torch

# Add sam3 to path assuming it is in the parent directory or ../sam3
sys.path.append('..') 
sys.path.append('../sam3')

try:
    from sam3.model_builder import build_sam3_image_model
    from sam3.model.sam3_image_processor import Sam3Processor
    print("SAM3 modules imported successfully.")
except ImportError as e:
    print(f"Error importing SAM3: {e}. Make sure the sam3 folder is correctly located.")

SAM3 modules imported successfully.


In [3]:
# Load the SAM3 Model
# This matches the logic from preprocessing/colab.ipynb
print("Loading SAM3 Model...")
model = build_sam3_image_model(enable_inst_interactivity=True)
processor = Sam3Processor(model)
print("Model loaded.")

Loading SAM3 Model...
Model loaded.


In [9]:
def preprocess_scanpath(parquet_path, dist_thresh=0.05, duration_thresh_ms=100):
    """
    Preprocesses scanpaths:
    1. Combines consecutive points if they are spatially close (dist < dist_thresh).
    2. Filters out points that have too low duration (duration < duration_thresh_ms).
    
    Args:
        parquet_path: Path to parquet file.
        dist_thresh: Normalized distance threshold (0-1) to merge consecutive points.
        duration_thresh_ms: Minimum duration in ms to keep a fixation.
    
    Returns:
        DataFrame with columns ['x', 'y', 'start_time', 'end_time', 'duration']
    """
    df = pd.read_parquet(parquet_path)
    if df.empty:
        return pd.DataFrame()
    
    # Extract arrays
    xs = df['x'].values
    ys = df['y'].values
    ds = df['duration_ms'].values
    
    fixations = []
    
    if len(xs) == 0:
        return pd.DataFrame()

    # Initial current group
    curr_x_sum = xs[0]
    curr_y_sum = ys[0]
    curr_count = 1
    duration = ds[0]
    timestart = 0
    # Estimate raw sample duration as avg diff or diff to next
    # For the last point, we'll assume same as prev interval or small default

    
    # Iterate
    for i in range(1, len(xs)):
        
            
        fixations.append({
            'x': curr_x_sum,
            'y': curr_y_sum,
            'start_time': timestart,
            'end_time': timestart + duration, # Approx end
            'duration': duration
        })

        timestart += duration
        
        # Start new group
        curr_x_sum = xs[i]
        curr_y_sum = ys[i]
        curr_count = 1
        duration = ds[i]

    fix_df = pd.DataFrame(fixations)
    
    # Filter by duration
    if not fix_df.empty:
        fix_df = fix_df[fix_df['duration'] >= duration_thresh_ms].reset_index(drop=True)
        
    return fix_df

print("Preprocessing function defined.")

Preprocessing function defined.


In [5]:
from tqdm import tqdm

def get_masks_for_fixations(image_path, fixations, processor):
    """
    Generates a mask for each fixation point using SAM3.
    """
    image_pil = Image.open(image_path).convert("RGB")
    width, height = image_pil.size
    
    # Process image once? SAM3 processor might handle caching, or we call it per prompt.
    # Looking at sam3 examples, usually we can pass points to the prompt.
    
    # Prepare inputs
    # processor expects image. 
    # For simplicity, we process row by row to allow "per point" segmentation selection
    
    all_masks = []
    
    print(f"Generating masks for {len(fixations)} fixations...")

    inference_state = processor.set_image(image_pil)

    for idx, row in tqdm(fixations.iterrows(), total=len(fixations), desc="Processing Fixations"):
        x_px = int(row['x'] * width)
        y_px = int(row['y'] * height)
        
        # Clip to image bounds
        x_px = max(0, min(width-1, x_px))
        y_px = max(0, min(height-1, y_px))
        
        # Construct prompt
        # SAM3 API: expects points like [[x, y]] and labels like [1]
        input_points = [[[x_px, y_px]]]
        input_labels = [[1]]
                
        masks, scores, logits = model.predict_inst(
            inference_state,
            point_coords=input_points,
            point_labels=input_labels,
            multimask_output=False,
        )
        sorted_ind = np.argsort(scores)[::-1]
        masks = masks[sorted_ind]
        scores = scores[sorted_ind]
        logits = logits[sorted_ind]
        
        ## 1. Standardize mask shape (handling 4D, 3D, or 2D inputs)
        if len(masks.shape) == 4:
            masks = masks.squeeze(0)
        if len(masks.shape) == 3 and masks.shape[0] == 1:
            masks = masks.squeeze(0)
        
        # 2. Combine all masks into one single master mask
        # We use np.max to ensure if masks overlap, they stay 'visible'
        if len(masks.shape) == 3:
            combined_mask = np.max(masks, axis=0)
        else:
            combined_mask = masks
        
        # Convert to numpy uint8 0 or 255
        # Masks are typically logits, so > 0 check converts to boolean
        mask_np = (combined_mask > 0).astype(np.uint8) * 255
        
        all_masks.append(mask_np)
            
    return all_masks

In [6]:
import os
from PIL import Image, ImageDraw
import cv2
import numpy as np

def save_fixation_images(image_path, fixations, masks, output_folder):
    # Ensure the output directory exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        print(f"Created directory: {output_folder}")

    # Load base image
    base_pil = Image.open(image_path).convert("RGBA")
    width, height = base_pil.size
    
    # Create the dimmed background
    overlay = Image.new("RGBA", base_pil.size, (0, 0, 0, 200))
    dimmed_bg_pil = Image.alpha_composite(base_pil, overlay)
    
    if fixations.empty:
        print("No fixations to process.")
        return

    print(f"Saving {len(fixations)} fixation images to: {output_folder}")
    radius = int(max(width, height) * 0.01)
    
    # Iterate through each fixation by its position in the sequence
    for i in range(len(fixations)):
        # 1. Prepare the background with the current mask
        frame_pil = dimmed_bg_pil.copy()
        
        if i < len(masks):
            mask_np = masks[i]
            if mask_np.shape[:2] != (height, width):
                mask_np = cv2.resize(mask_np, (width, height), interpolation=cv2.INTER_NEAREST)
            
            mask_pil = Image.fromarray(mask_np).convert("L")
            frame_pil = Image.composite(base_pil, frame_pil, mask_pil)
        
        # 2. Draw dots on a separate layer
        shapes_layer = Image.new("RGBA", (width, height), (0, 0, 0, 0))
        draw = ImageDraw.Draw(shapes_layer)

        # Draw all PREVIOUS fixations (Grey)
        for prev_idx in range(i):
            prev_row = fixations.iloc[prev_idx]
            px, py = prev_row['x'] * width, prev_row['y'] * height
            draw.ellipse((px - radius, py - radius, px + radius, py + radius), fill=(128, 128, 128, 100))
        
        # Draw the CURRENT fixation (Red)
        curr_row = fixations.iloc[i]
        cx, cy = curr_row['x'] * width, curr_row['y'] * height
        draw.ellipse((cx - radius, cy - radius, cx + radius, cy + radius), fill=(255, 0, 0, 200))
            
        # Composite dots onto the frame
        final_image = Image.alpha_composite(frame_pil, shapes_layer)
        
        # 3. Save with a filename based on the fixation sequence
        file_name = f"fixation_{str(i+1).zfill(3)}.png"
        save_path = os.path.join(output_folder, file_name)
        final_image.save(save_path)
        
        print(f"Saved image for fixation {i+1}/{len(fixations)}", end='\r')

    print(f"\nProcessing complete. Images saved in: {output_folder}")

# Usage
path = r"C:\Users\domin\OneDrive\Seminar\code\präsibilder"
# save_fixation_images(image_path, fixations_df, masks_list, path)

In [7]:
from PIL import Image, ImageDraw
import cv2
import numpy as np

def generate_scanpath_video(image_path, fixations, masks, output_video_path, fps=30):
    # Use PIL for image manipulation as requested
    base_pil = Image.open(image_path).convert("RGBA")
    width, height = base_pil.size
    
    # Create the background with the requested dark overlay (0,0,0,200)
    overlay = Image.new("RGBA", base_pil.size, (0, 0, 0, 200))
    dimmed_bg_pil = Image.alpha_composite(base_pil, overlay)
    
    if fixations.empty:
        print("No fixations to animate.")
        return

    min_time = fixations['start_time'].min()
    max_time = fixations['end_time'].max()
    duration_ms = max_time - min_time
    total_frames = int((duration_ms / 1000.0) * fps)
    
    # VideoWriter expects BGR, create it
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    print(f"Generating video: {output_video_path}")
    print(f"Duration: {duration_ms}ms, Frames: {total_frames}")
    
    # Dot radius (dynamic based on size, ~1.5% of max dimension)
    radius = int(max(width, height) * 0.01)
    
    for f in range(total_frames):
        current_time_ms = min_time + (f / fps) * 1000.0
        
        # Find active fixation
        active_fix = fixations[
            (fixations['start_time'] <= current_time_ms) & 
            (fixations['end_time'] >= current_time_ms)
        ]
        
        # Start matching active mask on dimmed
        frame_pil = dimmed_bg_pil.copy()
        
        if not active_fix.empty:
            idx = active_fix.index[0]
            if idx < len(masks):
                mask_np = masks[idx]
                
                # Resize if needed
                if mask_np.shape[:2] != (height, width):
                    mask_np = cv2.resize(mask_np, (width, height), interpolation=cv2.INTER_NEAREST)
                
                # Convert mask to PIL (L mode) for compositing
                mask_pil = Image.fromarray(mask_np).convert("L")
                
                # Composite: Show base_pil (bright) where mask is white, dimmed_bg elsewhere
                frame_pil = Image.composite(base_pil, frame_pil, mask_pil)
        
        # Overlay drawing layer (for dots and lines with proper alpha blending)
        shapes_layer = Image.new("RGBA", (width, height), (0, 0, 0, 0))
        draw = ImageDraw.Draw(shapes_layer)

        # Collect points for the path (all fixations started up to now)
        # Includes finished grey ones and potentially the active red one.
        # "Connect all grey and the red dot"
        
        # 1. Previous fixations (Grey)
        prev_fixs = fixations[fixations['end_time'] < current_time_ms]
        for _, row in prev_fixs.iterrows():
            px, py = row['x'] * width, row['y'] * height
            draw.ellipse((px - radius, py - radius, px + radius, py + radius), fill=(128, 128, 128, 100))
        
        # 2. Current fixation (Red)
        if not active_fix.empty:
            row = active_fix.iloc[0]
            px, py = row['x'] * width, row['y'] * height
            draw.ellipse((px - radius, py - radius, px + radius, py + radius), fill=(255, 0, 0, 200))
            
        # Composite shapes onto frame
        frame_pil = Image.alpha_composite(frame_pil, shapes_layer)
        
        # Convert PIL RGBA -> RGB -> BGR for OpenCV
        frame_rgb = frame_pil.convert("RGB")
        frame_bgr = np.array(frame_rgb)[:, :, ::-1]
        
        out.write(frame_bgr)
        
        if f % 30 == 0:
            print(f"Frame {f}/{total_frames}", end='\r')

    out.release()
    print("\nVideo saved.")

In [10]:
import glob
import os

# --- BATCH PROCESSING PIPELINE ---

# Configuration
# Images are in ../test_images
# Parquet files are in ../parquet
# Output should be in ../test_videos_clean

IMAGE_FOLDER = r'../test_images'
PARQUET_FOLDER = r'../parquet'
OUTPUT_FOLDER = r'../test_videos_clean'
PARTICIPANT_ID = 'Proband1'  # Updated for new naming convention

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
    
    print(f"Created output directory: {OUTPUT_FOLDER}")

# Get all jpg and png images in test_images
# "It should also scan for png"
image_files = glob.glob(os.path.join(IMAGE_FOLDER, "*.jpg")) + glob.glob(os.path.join(IMAGE_FOLDER, "*.png"))
print(f"Found {len(image_files)} images (jpg+png) in {IMAGE_FOLDER}")
print("-" * 50)

for img_path in image_files:
    img_name = os.path.basename(img_path)
    # Extract Image ID from filename (e.g. "01.jpg" -> "01")
    img_id = os.path.splitext(img_name)[0]
    
    # Construct search pattern for parquet file
    # New Format: Proband1_..._01.parquet
    # We search for files starting with PARTICIPANT_ID and ending with _{img_id}.parquet
    search_pattern = os.path.join(PARQUET_FOLDER, f"{PARTICIPANT_ID}_{img_id}.parquet")
    found_files = glob.glob(search_pattern)
    
    # Check if corresponding parquet file exists
    if not found_files:
        print(f"[MISSING] Parquet file not found for {img_name} (ID: {img_id})")
        print(f"          Expected pattern: {search_pattern}")
        continue
        
    # Take the first match
    parquet_path = found_files[0]
    parquet_name = os.path.basename(parquet_path)
        
    print(f"\n[PROCESSING] {img_name} -> {parquet_name}")
    
    # 1. Preprocess Scanpath
    # Using thresholds from previous cells
    fixations_df = preprocess_scanpath(parquet_path, dist_thresh=0.05, duration_thresh_ms=80)
    if fixations_df.empty:
        print(f"   [SKIP] No valid fixations found after preprocessing.")
        continue
        
    print(f"   Fixations: {len(fixations_df)}")

    # 2. Extract Masks
    # Determine masks using the global 'processor' and 'model'
    try:
        masks = get_masks_for_fixations(img_path, fixations_df, processor)
    except Exception as e:
        print(f"   [ERROR] Failed to extract masks: {e}")
        continue
        
    # 3. Generate Video
    video_name = parquet_name.replace('.parquet', '.mp4')
    output_video_path = os.path.join(OUTPUT_FOLDER, video_name)
    
    try:
        generate_scanpath_video(img_path, fixations_df, masks, output_video_path, fps=30)
        print(f"   [SUCCESS] Saved to {output_video_path}")
    except Exception as e:
        print(f"   [ERROR] Failed to generate video: {e}")

print("\n" + "=" * 50)
print("Batch processing complete.")

Found 20 images (jpg+png) in ../test_images
--------------------------------------------------

[PROCESSING] 01.jpg -> Proband1_01.parquet
   Fixations: 22
Generating masks for 22 fixations...


Processing Fixations: 100%|██████████| 22/22 [00:04<00:00,  4.68it/s]


Generating video: ../test_videos_clean\Proband1_01.mp4
Duration: 6683.333333333333ms, Frames: 200
Frame 180/200
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband1_01.mp4

[PROCESSING] 02.jpg -> Proband1_02.parquet
   Fixations: 25
Generating masks for 25 fixations...


Processing Fixations: 100%|██████████| 25/25 [01:38<00:00,  3.93s/it]


Generating video: ../test_videos_clean\Proband1_02.mp4
Duration: 6016.666666666668ms, Frames: 180
Frame 150/180
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband1_02.mp4

[PROCESSING] 03.jpg -> Proband1_03.parquet
   Fixations: 28
Generating masks for 28 fixations...


...

Generating video: ../test_videos_clean\Proband1_12.mp4
Duration: 5216.666666666669ms, Frames: 156
Frame 150/156
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband1_12.mp4

[PROCESSING] 17.png -> Proband1_17.parquet
   Fixations: 22
Generating masks for 22 fixations...


Processing Fixations: 100%|██████████| 22/22 [01:53<00:00,  5.14s/it]


Generating video: ../test_videos_clean\Proband1_17.mp4
Duration: 6750.000000000001ms, Frames: 202
Frame 180/202
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband1_17.mp4

[PROCESSING] 20.png -> Proband1_20.parquet
   Fixations: 23
Generating masks for 23 fixations...


Processing Fixations: 100%|██████████| 23/23 [01:50<00:00,  4.82s/it]


Generating video: ../test_videos_clean\Proband1_20.mp4
Duration: 6533.333333333331ms, Frames: 195
Frame 180/195
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband1_20.mp4

Batch processing complete.


In [11]:
import glob
import os

# --- BATCH PROCESSING PIPELINE ---

# Configuration
# Images are in ../test_images
# Parquet files are in ../parquet
# Output should be in ../test_videos_clean

IMAGE_FOLDER = r'../test_images'
PARQUET_FOLDER = r'../parquet'
OUTPUT_FOLDER = r'../test_videos_clean'
PARTICIPANT_ID = 'Proband2'  # Updated for new naming convention

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
    
    print(f"Created output directory: {OUTPUT_FOLDER}")

# Get all jpg and png images in test_images
# "It should also scan for png"
image_files = glob.glob(os.path.join(IMAGE_FOLDER, "*.jpg")) + glob.glob(os.path.join(IMAGE_FOLDER, "*.png"))
print(f"Found {len(image_files)} images (jpg+png) in {IMAGE_FOLDER}")
print("-" * 50)

for img_path in image_files:
    img_name = os.path.basename(img_path)
    # Extract Image ID from filename (e.g. "01.jpg" -> "01")
    img_id = os.path.splitext(img_name)[0]
    
    # Construct search pattern for parquet file
    # New Format: Proband1_..._01.parquet
    # We search for files starting with PARTICIPANT_ID and ending with _{img_id}.parquet
    search_pattern = os.path.join(PARQUET_FOLDER, f"{PARTICIPANT_ID}_{img_id}.parquet")
    found_files = glob.glob(search_pattern)
    
    # Check if corresponding parquet file exists
    if not found_files:
        print(f"[MISSING] Parquet file not found for {img_name} (ID: {img_id})")
        print(f"          Expected pattern: {search_pattern}")
        continue
        
    # Take the first match
    parquet_path = found_files[0]
    parquet_name = os.path.basename(parquet_path)
        
    print(f"\n[PROCESSING] {img_name} -> {parquet_name}")
    
    # 1. Preprocess Scanpath
    # Using thresholds from previous cells
    fixations_df = preprocess_scanpath(parquet_path, dist_thresh=0.05, duration_thresh_ms=80)
    
    if fixations_df.empty:
        print(f"   [SKIP] No valid fixations found after preprocessing.")
        continue
        
    print(f"   Fixations: {len(fixations_df)}")

    # 2. Extract Masks
    # Determine masks using the global 'processor' and 'model'
    try:
        masks = get_masks_for_fixations(img_path, fixations_df, processor)
    except Exception as e:
        print(f"   [ERROR] Failed to extract masks: {e}")
        continue
        
    # 3. Generate Video
    video_name = parquet_name.replace('.parquet', '.mp4')
    output_video_path = os.path.join(OUTPUT_FOLDER, video_name)
    
    try:
        generate_scanpath_video(img_path, fixations_df, masks, output_video_path, fps=30)
        print(f"   [SUCCESS] Saved to {output_video_path}")
    except Exception as e:
        print(f"   [ERROR] Failed to generate video: {e}")

print("\n" + "=" * 50)
print("Batch processing complete.")

Found 20 images (jpg+png) in ../test_images
--------------------------------------------------

[PROCESSING] 01.jpg -> Proband2_01.parquet
   Fixations: 14
Generating masks for 14 fixations...


Processing Fixations: 100%|██████████| 14/14 [01:46<00:00,  7.58s/it]


Generating video: ../test_videos_clean\Proband2_01.mp4
Duration: 7083.333333333334ms, Frames: 212
Frame 210/212
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband2_01.mp4

[PROCESSING] 02.jpg -> Proband2_02.parquet
   Fixations: 9
Generating masks for 9 fixations...


Processing Fixations: 100%|██████████| 9/9 [01:45<00:00, 11.70s/it] 


Generating video: ../test_videos_clean\Proband2_02.mp4
Duration: 7283.333333333335ms, Frames: 218
Frame 210/218
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband2_02.mp4

[PROCESSING] 03.jpg -> Proband2_03.parquet
   Fixations: 20
Generating masks for 20 fixations...


...

Generating video: ../test_videos_clean\Proband2_12.mp4
Duration: 4866.66666666667ms, Frames: 146
Frame 120/146
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband2_12.mp4

[PROCESSING] 17.png -> Proband2_17.parquet
   Fixations: 17
Generating masks for 17 fixations...


Processing Fixations: 100%|██████████| 17/17 [01:45<00:00,  6.20s/it]


Generating video: ../test_videos_clean\Proband2_17.mp4
Duration: 6383.333333333333ms, Frames: 191
Frame 180/191
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband2_17.mp4

[PROCESSING] 20.png -> Proband2_20.parquet
   Fixations: 13
Generating masks for 13 fixations...


Processing Fixations: 100%|██████████| 13/13 [01:43<00:00,  8.00s/it]


Generating video: ../test_videos_clean\Proband2_20.mp4
Duration: 5850.0ms, Frames: 175
Frame 150/175
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband2_20.mp4

Batch processing complete.


In [12]:
import glob
import os

# --- BATCH PROCESSING PIPELINE ---

# Configuration
# Images are in ../test_images
# Parquet files are in ../parquet
# Output should be in ../test_videos_clean

IMAGE_FOLDER = r'../test_images'
PARQUET_FOLDER = r'../parquet'
OUTPUT_FOLDER = r'../test_videos_clean'
PARTICIPANT_ID = 'Proband3'  # Updated for new naming convention

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)
    
    print(f"Created output directory: {OUTPUT_FOLDER}")

# Get all jpg and png images in test_images
# "It should also scan for png"
image_files = glob.glob(os.path.join(IMAGE_FOLDER, "*.jpg")) + glob.glob(os.path.join(IMAGE_FOLDER, "*.png"))
print(f"Found {len(image_files)} images (jpg+png) in {IMAGE_FOLDER}")
print("-" * 50)

for img_path in image_files:
    img_name = os.path.basename(img_path)
    # Extract Image ID from filename (e.g. "01.jpg" -> "01")
    img_id = os.path.splitext(img_name)[0]
    
    # Construct search pattern for parquet file
    # New Format: Proband1_..._01.parquet
    # We search for files starting with PARTICIPANT_ID and ending with _{img_id}.parquet
    search_pattern = os.path.join(PARQUET_FOLDER, f"{PARTICIPANT_ID}_{img_id}.parquet")
    found_files = glob.glob(search_pattern)
    
    # Check if corresponding parquet file exists
    if not found_files:
        print(f"[MISSING] Parquet file not found for {img_name} (ID: {img_id})")
        print(f"          Expected pattern: {search_pattern}")
        continue
        
    # Take the first match
    parquet_path = found_files[0]
    parquet_name = os.path.basename(parquet_path)
        
    print(f"\n[PROCESSING] {img_name} -> {parquet_name}")
    
    # 1. Preprocess Scanpath
    # Using thresholds from previous cells
    fixations_df = preprocess_scanpath(parquet_path, dist_thresh=0.05, duration_thresh_ms=80)
    
    if fixations_df.empty:
        print(f"   [SKIP] No valid fixations found after preprocessing.")
        continue
        
    print(f"   Fixations: {len(fixations_df)}")

    # 2. Extract Masks
    # Determine masks using the global 'processor' and 'model'
    try:
        masks = get_masks_for_fixations(img_path, fixations_df, processor)
    except Exception as e:
        print(f"   [ERROR] Failed to extract masks: {e}")
        continue
        
    # 3. Generate Video
    video_name = parquet_name.replace('.parquet', '.mp4')
    output_video_path = os.path.join(OUTPUT_FOLDER, video_name)
    
    try:
        generate_scanpath_video(img_path, fixations_df, masks, output_video_path, fps=30)
        print(f"   [SUCCESS] Saved to {output_video_path}")
    except Exception as e:
        print(f"   [ERROR] Failed to generate video: {e}")

print("\n" + "=" * 50)
print("Batch processing complete.")

Found 20 images (jpg+png) in ../test_images
--------------------------------------------------

[PROCESSING] 01.jpg -> Proband3_01.parquet
   Fixations: 14
Generating masks for 14 fixations...


Processing Fixations: 100%|██████████| 14/14 [01:44<00:00,  7.46s/it]


Generating video: ../test_videos_clean\Proband3_01.mp4
Duration: 6666.666666666668ms, Frames: 200
Frame 180/200
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband3_01.mp4

[PROCESSING] 02.jpg -> Proband3_02.parquet
   Fixations: 25
Generating masks for 25 fixations...


Processing Fixations: 100%|██████████| 25/25 [01:46<00:00,  4.25s/it]


Generating video: ../test_videos_clean\Proband3_02.mp4
Duration: 6133.333333333335ms, Frames: 184
Frame 180/184
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband3_02.mp4

[PROCESSING] 03.jpg -> Proband3_03.parquet
   Fixations: 23
Generating masks for 23 fixations...


...

Generating video: ../test_videos_clean\Proband3_12.mp4
Duration: 5916.66666666667ms, Frames: 177
Frame 150/177
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband3_12.mp4

[PROCESSING] 17.png -> Proband3_17.parquet
   Fixations: 15
Generating masks for 15 fixations...


Processing Fixations: 100%|██████████| 15/15 [01:45<00:00,  7.00s/it]


Generating video: ../test_videos_clean\Proband3_17.mp4
Duration: 6083.333333333336ms, Frames: 182
Frame 180/182
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband3_17.mp4

[PROCESSING] 20.png -> Proband3_20.parquet
   Fixations: 28
Generating masks for 28 fixations...


Processing Fixations: 100%|██████████| 28/28 [01:47<00:00,  3.83s/it]


Generating video: ../test_videos_clean\Proband3_20.mp4
Duration: 6300.0ms, Frames: 189
Frame 180/189
Video saved.
   [SUCCESS] Saved to ../test_videos_clean\Proband3_20.mp4

Batch processing complete.


In [ ]:
prompt_scanpath = """
Role: You are an expert Visual Cognitive Scientist and Multimedia Analyst.

Task: Analyze the provided eye-tracking video to reconstruct the user's "Visual Journey." The video contains a greyed-out background image (the original stimulus) with a dynamic red dot representing the gaze point. Areas of focus are highlighted via Segmented Anything Model (SAM) cutouts that appear over the gaze point.

Analysis Workflow:
- Contextual Reconstruction: Identify the underlying social media image. What is the primary content (e.g., an advertisement, a personal photo, an infographic)?
- Scanpath Mapping: Describe the movement of the red dot. Does it follow a predictable pattern (like an F-pattern or Z-pattern)? Note the duration of "dwell times" on specific SAM-segmented objects.
- Semantic Interpretation: For every SAM-segmented cutout that appears:
    - Identify the object.
    - Explain its significance within the image's composition.
- Preference Inference: Based on dwell time, repeated fixations (re-visitation), and the order of scanning, determine which element the user was "most fond of" or found most engaging. Provide a confidence score for this inference.

Output Format:
- Stimulus Overview: (Summary of the background image)
- Behavioral Analysis: (Interpretation of the user's intent—e.g., "The user skipped the text and focused immediately on the product's price tag.")
- Key Insight: Identify the "Hero Element" (the object of highest interest).

Do not output the confidence score. no bold or italic formatting. no emojis.
"""
from google import genai
import time
import glob
import os

# Initialize the client
client = genai.Client(api_key="Your-API-Key-Here")

video_scanpath_folder = r'../test_videos_clean/Simon'
outputs_scanpath_folder = r'../test_outputs/scanpath/P3'

# Get all jpg images in test_images
video_scanpath_files = glob.glob(os.path.join(video_scanpath_folder, "*.mp4"))
print(f"Found {len(video_scanpath_files)} videos in {video_scanpath_folder}")
print("-" * 50)

for video_path in video_scanpath_files:
    # 1. Upload the video
    # This is required for videos larger than 20MB or longer than 1 minute
    print("Uploading video...")
    video_file = client.files.upload(file=video_path)

    # 2. Wait for processing
    # Video files must be in 'ACTIVE' state before they can be used in a prompt
    while video_file.state.name == "PROCESSING":
        print("Processing...", end="\r")
        time.sleep(2)
        video_file = client.files.get(name=video_file.name)

    print("\nVideo is ready!")

    # 3. Prompt with Text + Video
    response = client.models.generate_content(
        model="gemini-2.5-pro", 
        contents=[
            video_file,
            prompt_scanpath
        ]
    )

    # 4. Save the response in a text file in test_outputs/scanpath
    output_text_path = os.path.join(outputs_scanpath_folder, os.path.basename(video_path).replace('.mp4', '_scanpath.txt'))
    with open(output_text_path, 'w') as f:
        f.write(response.text)
    print(f"Response saved to {output_text_path}")

    # 4. Cleanup (Optional)
    # Files are auto-deleted after 48 hours, but you can delete manually
    resp = client.files.delete(name=video_file.name)

print("\n" + "=" * 50)
print("All video analyses complete.")

Found 20 videos in ../test_videos_clean/Simon
--------------------------------------------------
Uploading video...
Processing...
Video is ready!
Response saved to ../test_outputs/scanpath/P3\Proband3_01_scanpath.txt
Uploading video...
Processing...
Video is ready!
Response saved to ../test_outputs/scanpath/P3\Proband3_02_scanpath.txt
Uploading video...
Processing...
Video is ready!
Response saved to ../test_outputs/scanpath/P3\Proband3_03_scanpath.txt
Uploading video...
Processing...
Video is ready!
Response saved to ../test_outputs/scanpath/P3\Proband3_04_scanpath.txt
Uploading video...
Processing...
Video is ready!
Response saved to ../test_outputs/scanpath/P3\Proband3_05_scanpath.txt
Uploading video...
Processing...
Video is ready!
Response saved to ../test_outputs/scanpath/P3\Proband3_06_scanpath.txt
Uploading video...
Processing...
Video is ready!
Response saved to ../test_outputs/scanpath/P3\Proband3_07_scanpath.txt
Uploading video...
Processing...
Video is ready!
Response saved